<a href="https://colab.research.google.com/github/mavillan/jane-street-market-prediction/blob/main/tuning/bottlenet-optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -Iv scikit-learn==0.23.2 > /dev/null 2>&1
!pip install optuna > /dev/null 2>&1

In [3]:
import copy
import os
import numpy as np 
import pandas as pd
from pathlib import Path
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

import torch
import torch.nn as nn
from torch.utils.data import Dataset,TensorDataset,DataLoader

# custom modules
import sys
sys.path.append("/content/drive/MyDrive/kaggle/janestreet")
from torch_utils import Monitor, train_step, valid_step

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
set_seed(2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

***
### utility functions

In [4]:
def utility_score(date, weight, resp, action):
    """
    Fast computation of utility score
    """
    date = date.astype(int)
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return -u

In [5]:
def cat_encoder(X):
    """
    Fast one-hot encoding of feature_0
    """
    X["feature_00"] = 0
    idx00 = X.query("feature_0 == -1").index
    X.loc[idx00,"feature_00"] = 1
    
    X["feature_01"] = 0
    idx01 = X.query("feature_0 == 1").index
    X.loc[idx01,"feature_01"] = 1
    
    return X.iloc[:,1:]

***
### preparing the data

In [6]:
root = Path("/content/drive/MyDrive/kaggle/janestreet/preprocessing/")

train = pd.read_parquet(root/"train.parquet")
features = pd.read_parquet(root/"features.parquet")

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390491 entries, 0 to 2390490
Columns: 143 entries, date to w4
dtypes: float32(140), int16(1), int32(1), int8(1)
memory usage: 1.3 GB


In [7]:
train = train.query("date > 85").query("weight > 0").reset_index(drop=True)

input_features = [col for col in train.columns if "feature" in col]
resp_cols = ['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']
w_cols = ["w", "w1", "w2", "w3", "w4"]

X_dset = train.loc[:,input_features].copy()
y_dset = (train.loc[:,resp_cols] > 0).astype(int).copy()
w_dset = train.loc[:, w_cols].copy()
dwr_dset = train.loc[:, ["date","weight","resp"]].copy()

In [8]:
%%time 

with open("/content/drive/MyDrive/kaggle/janestreet/imputer/imputer_f0m1.pickle", "rb") as file:
    imputer_f0m1 = pickle.load(file)
    file.close()
    
with open("/content/drive/MyDrive/kaggle/janestreet/imputer/imputer_f0p1.pickle", "rb") as file:
    imputer_f0p1 = pickle.load(file)
    file.close()

idx_f0m1 = X_dset.query("feature_0 == -1").index
X_dset.loc[idx_f0m1, input_features[1:]] = imputer_f0m1.transform(X_dset.loc[idx_f0m1, input_features[1:]])

idx_f0p1 = X_dset.query("feature_0 ==  1").index
X_dset.loc[idx_f0p1, input_features[1:]] = imputer_f0p1.transform(X_dset.loc[idx_f0p1, input_features[1:]])

CPU times: user 45.2 s, sys: 6.91 s, total: 52.1 s
Wall time: 47.9 s


In [9]:
X_dset = cat_encoder(X_dset)
input_features = X_dset.columns.tolist()

***
### model definition

In [10]:
class GBN(nn.Module):
    """
    Ghost Batch Normalization
    https://arxiv.org/abs/1705.08741
    """

    def __init__(self, input_dim, virtual_batch_size=128, momentum=0.01):
        super(GBN, self).__init__()

        self.input_dim = input_dim
        self.virtual_batch_size = virtual_batch_size
        self.bn = nn.BatchNorm1d(self.input_dim, momentum=momentum)

    def forward(self, x):
        chunks = x.chunk(int(np.ceil(x.shape[0] / self.virtual_batch_size)), 0)
        res = [self.bn(x_) for x_ in chunks]

        return torch.cat(res, dim=0)
    
class NormalLinear(nn.Module):
    """ 
    Linear layer with normalized weights
    """
    def __init__(self, size_in, size_out, bias=True):
        super(NormalLinear, self).__init__()
        self.size_in, self.size_out = size_in, size_out
        # weights vector
        weights_v = torch.Tensor(size_out, size_in)
        nn.init.kaiming_uniform_(weights_v, a=np.sqrt(5)) 
        self.weights_v = nn.Parameter(weights_v)
        # weights magnitude
        weights_m = torch.norm(weights_v, dim=1, keepdim=True)
        self.weights_m = nn.Parameter(weights_m.clone().detach())
        
        if bias:
            bias_v = torch.Tensor(size_out)    
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(weights_v)
            bound = 1 / np.sqrt(fan_in)
            nn.init.uniform_(bias_v, -bound, bound)
            self.bias = nn.Parameter(bias_v)
        else:
            self.register_parameter('bias', None)
            
    def _compute_weights(self):
        norm_per_output = torch.norm(self.weights_v, dim=1, keepdim=True)
        return self.weights_m * torch.div(self.weights_v, norm_per_output)
            
    def forward(self, x):
        weights = self._compute_weights()
        return nn.functional.linear(x, weights, self.bias)

In [11]:
class BottleNet(nn.Module):
    """
    NN with bottleneck input
    """

    def __init__(self, input_dim, output_dim, encode_dim, nn_depth, nn_width, 
                 dropout_ae=0.1, dropout_input=0.2, dropout_hidden=0.3, 
                 dropout_output=0.1, momentum=0.02, virtual_batch_size=128):
        super().__init__()
        
        self.bn_encoder = GBN(input_dim, virtual_batch_size=virtual_batch_size, momentum=momentum)
        self.dp_encoder = nn.Dropout(dropout_ae)
        self.ln_encoder = NormalLinear(input_dim, encode_dim, bias=False)
        
        self.bn_decoder = GBN(encode_dim, virtual_batch_size=virtual_batch_size, momentum=momentum)
        self.dp_decoder = nn.Dropout(dropout_ae)
        self.ln_decoder = NormalLinear(encode_dim, input_dim, bias=False)
        
        self.bn_in = GBN(encode_dim, virtual_batch_size=virtual_batch_size, momentum=momentum)
        self.dp_in = nn.Dropout(dropout_input)
        self.ln_in = NormalLinear(encode_dim, nn_width, bias=False)
        
        self.bnorms = nn.ModuleList(
            [GBN(nn_width, virtual_batch_size=virtual_batch_size, momentum=momentum) 
             for i in range(nn_depth-1)])
        self.dropouts = nn.ModuleList(
            [nn.Dropout(dropout_hidden) 
             for i in range(nn_depth-1)])
        self.linears = nn.ModuleList(
            [NormalLinear(nn_width, nn_width, bias=False) 
             for i in range(nn_depth-1)])
        
        self.bn_out = GBN(nn_width, virtual_batch_size=virtual_batch_size, momentum=momentum)
        self.dp_out = nn.Dropout(dropout_output)
        self.ln_out = NormalLinear(nn_width, output_dim, bias=False)
        
    def encoder(self, x):
        x = self.bn_encoder(x)
        x = self.dp_encoder(x)
        x = self.ln_encoder(x)
        x = nn.functional.relu(x)
        return x
    
    def decoder(self, x):
        x = self.bn_decoder(x)
        x = self.dp_decoder(x)
        x = self.ln_decoder(x)
        x = nn.functional.relu(x)
        return x

    def forward(self, x):
        x_encoded = self.encoder(x)
        x_decoded = self.decoder(x_encoded)
        
        x = self.bn_in(x_encoded)
        x = self.dp_in(x)
        x = self.ln_in(x)
        x = nn.functional.relu(x)

        for bn_layer,dp_layer,ln_layer in zip(self.bnorms,self.dropouts,self.linears):
            x = bn_layer(x)
            x = dp_layer(x)
            x = ln_layer(x)
            x = nn.functional.relu(x)
            
        x = self.bn_out(x)
        x = self.dp_out(x)
        x = self.ln_out(x)
        
        return x,x_decoded

In [12]:
class BCELabelSmoothing(nn.Module):
    def __init__(self, label_smoothing=0.0):
        super(BCELabelSmoothing, self).__init__()
        self.label_smoothing = label_smoothing
        self.bce_loss = torch.nn.functional.binary_cross_entropy_with_logits
        
    def forward(self, prediction, target, weight=None):
        target_smooth = target*(1.0 - self.label_smoothing) + 0.5*self.label_smoothing
        if weight is None:
            loss = self.bce_loss(prediction, target_smooth, reduction="mean")
        else:
            loss = self.bce_loss(prediction, target_smooth, weight, reduction="sum") / torch.sum(weight)
        return loss

bce_loss = BCELabelSmoothing(label_smoothing=1e-2)

***
### Hyperparameter tuning

In [13]:
# 80% rows for train & 20% for valid
train_idx = train.query("date < 430").index
valid_idx = train.query("date >= 430").index

train_dset = TensorDataset(torch.tensor(X_dset.loc[train_idx].values, dtype=torch.float), 
                           torch.tensor(y_dset.loc[train_idx].values, dtype=torch.float),
                           torch.tensor(w_dset.loc[train_idx].values, dtype=torch.float),
                           torch.tensor(dwr_dset.loc[train_idx].values, dtype=torch.float),
                          )

valid_dset = TensorDataset(torch.tensor(X_dset.loc[valid_idx].values, dtype=torch.float), 
                           torch.tensor(y_dset.loc[valid_idx].values, dtype=torch.float),
                           torch.tensor(w_dset.loc[valid_idx].values, dtype=torch.float),
                           torch.tensor(dwr_dset.loc[valid_idx].values, dtype=torch.float),
                          )

dataset_sizes = {'train': len(train_dset), 'valid': len(valid_dset)}
train_dataloader = DataLoader(train_dset, batch_size=2048, shuffle=True, num_workers=2)
valid_dataloader = DataLoader(valid_dset, batch_size=len(valid_dset), shuffle=False, num_workers=2)

print("Number of step per epoch:", len(train_dset)//2048)

Number of step per epoch: 612


In [14]:
if os.path.exists(f"/content/drive/MyDrive/kaggle/janestreet/logs/bottlenet_tuning.csv"):
    logger = open(f"/content/drive/MyDrive/kaggle/janestreet/logs/bottlenet_tuning.csv", "a")
else:
    logger = open(f"/content/drive/MyDrive/kaggle/janestreet/logs/bottlenet_tuning.csv", "w")
    logger.write("trial;params;loss;metric;loss_hist;metric_hist\n")

In [15]:
default_nn_kwargs = {
    "input_dim":len(input_features),
    "output_dim":len(resp_cols),
    "nn_depth":3,
    "virtual_batch_size":256,
    "momentum":0.1,
    }

def objective(trial):
    sampled_nn_kwargs = {
        "encode_dim": int(trial.suggest_discrete_uniform("encode_dim", 64, 80, 8)),
        "nn_width": int(trial.suggest_discrete_uniform("nn_width", 96, 112, 16)),
        "dropout_ae": trial.suggest_discrete_uniform("dropout_ae", 0.1, 0.3, 0.05),
        "dropout_input": trial.suggest_discrete_uniform("dropout_input", 0.1, 0.3, 0.05),
        "dropout_hidden": trial.suggest_discrete_uniform("dropout_hidden", 0.2, 0.4, 0.05),
        "dropout_output": trial.suggest_discrete_uniform("dropout_output", 0.1, 0.2, 0.05),
        }
    nn_kwargs = {**sampled_nn_kwargs, **default_nn_kwargs}

    # other hyperparams
    weight_decay = 10 ** trial.suggest_int("weight_decay", -5, -4)
    pct_start = trial.suggest_discrete_uniform("pct_start", 0.2, 0.5, 0.1)
    max_lr = trial.suggest_discrete_uniform("max_lr", 0.1, 1.0, 0.1) * 1e-1

    sampled_params = {
        **sampled_nn_kwargs,
        "weight_decay":weight_decay,
        "pct_start":pct_start,
        "max_lr":max_lr,
        }
    print("-"*80)
    print("sampled_params:", sampled_params)

    model = BottleNet(**nn_kwargs)
    model = model.to(device)

    # 1st step: trains the autoencoder + snn jointly
    optimizer = torch.optim.SGD(model.parameters(), lr=max_lr/10, momentum=0.9, weight_decay=weight_decay)

    monitor = Monitor(
        model=model,
        optimizer=optimizer,
        scheduler=None,
        patience=5,
        metric_fn=utility_score,
        experiment_name=f'BottleNet',
        num_epochs=20,
        dataset_sizes=dataset_sizes,
        early_stop_on_metric=False,
        lower_is_better=True)

    for epoch in monitor.iter_epochs:
        train_step(model, train_dataloader, optimizer, monitor, bce_loss, dae=True, alpha=0.5)    
        early_stop = valid_step(model, valid_dataloader, optimizer, monitor, bce_loss, dae=True, alpha=0.5)
        if early_stop: break

    # 2nd step: trains snn only
    model.load_state_dict(monitor.best_model_state)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=weight_decay)

    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=max_lr,
        epochs=50,
        pct_start=pct_start, 
        anneal_strategy='cos', 
        cycle_momentum=True, 
        base_momentum=0.8, 
        max_momentum=0.9, 
        div_factor=1e1,
        final_div_factor=1e0,
        steps_per_epoch=len(train_dataloader),
        verbose=False)
    
    monitor = Monitor(
        model=model,
        optimizer=optimizer,
        scheduler=scheduler,
        patience=10,
        metric_fn=utility_score,
        experiment_name=f'BottleNet',
        num_epochs=50,
        dataset_sizes=dataset_sizes,
        early_stop_on_metric=False,
        lower_is_better=True)
    
    for epoch in monitor.iter_epochs:
        train_step(model, train_dataloader, optimizer, monitor, bce_loss, scheduler=scheduler, dae=True, alpha=0.)    
        early_stop = valid_step(model, valid_dataloader, optimizer, monitor, bce_loss, dae=True, alpha=0.)
        if early_stop: break

    logger.write(f"{trial.number};{sampled_params};{monitor.best_loss};{monitor.best_metric};{monitor.valid_loss[-10:]};{monitor.valid_metric[-10:]}\n")
    logger.flush()

    print(f"best_valid_loss: {monitor.best_loss} - best_valid_metric: {monitor.best_metric}")
    return monitor.best_loss

In [16]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200, timeout=43200, show_progress_bar=False)

[I 2021-02-21 05:17:29,974] A new study created in memory with name: no-name-2e352e23-6d5c-473d-b598-363fba4f7556


--------------------------------------------------------------------------------
sampled_params: {'encode_dim': 72, 'nn_width': 112, 'dropout_ae': 0.3, 'dropout_input': 0.1, 'dropout_hidden': 0.35000000000000003, 'dropout_output': 0.15000000000000002, 'weight_decay': 0.0001, 'pct_start': 0.5, 'max_lr': 0.020000000000000004}


[I 2021-02-21 06:05:06,157] Trial 0 finished with value: 0.6856082081794739 and parameters: {'encode_dim': 72.0, 'nn_width': 112.0, 'dropout_ae': 0.3, 'dropout_input': 0.1, 'dropout_hidden': 0.35000000000000003, 'dropout_output': 0.15000000000000002, 'weight_decay': -4, 'pct_start': 0.5, 'max_lr': 0.2}. Best is trial 0 with value: 0.6856082081794739.



best_valid_loss: 0.6856082081794739 - best_valid_metric: -2435.3428808027365
--------------------------------------------------------------------------------
sampled_params: {'encode_dim': 64, 'nn_width': 112, 'dropout_ae': 0.25, 'dropout_input': 0.1, 'dropout_hidden': 0.2, 'dropout_output': 0.1, 'weight_decay': 1e-05, 'pct_start': 0.30000000000000004, 'max_lr': 0.05}


[I 2021-02-21 06:41:57,133] Trial 1 finished with value: 0.6858002543449402 and parameters: {'encode_dim': 64.0, 'nn_width': 112.0, 'dropout_ae': 0.25, 'dropout_input': 0.1, 'dropout_hidden': 0.2, 'dropout_output': 0.1, 'weight_decay': -5, 'pct_start': 0.30000000000000004, 'max_lr': 0.5}. Best is trial 0 with value: 0.6856082081794739.


...


best_valid_loss: 0.6857430934906006 - best_valid_metric: -2653.5145269998034
--------------------------------------------------------------------------------
sampled_params: {'encode_dim': 72, 'nn_width': 112, 'dropout_ae': 0.15000000000000002, 'dropout_input': 0.25, 'dropout_hidden': 0.2, 'dropout_output': 0.1, 'weight_decay': 0.0001, 'pct_start': 0.2, 'max_lr': 0.06}


[I 2021-02-21 16:52:55,136] Trial 18 finished with value: 0.6853654384613037 and parameters: {'encode_dim': 72.0, 'nn_width': 112.0, 'dropout_ae': 0.15000000000000002, 'dropout_input': 0.25, 'dropout_hidden': 0.2, 'dropout_output': 0.1, 'weight_decay': -4, 'pct_start': 0.2, 'max_lr': 0.6}. Best is trial 13 with value: 0.6851402521133423.



best_valid_loss: 0.6853654384613037 - best_valid_metric: -2645.4066745254536
--------------------------------------------------------------------------------
sampled_params: {'encode_dim': 64, 'nn_width': 96, 'dropout_ae': 0.1, 'dropout_input': 0.15000000000000002, 'dropout_hidden': 0.25, 'dropout_output': 0.1, 'weight_decay': 1e-05, 'pct_start': 0.30000000000000004, 'max_lr': 0.04000000000000001}


[I 2021-02-21 17:32:11,870] Trial 19 finished with value: 0.685600221157074 and parameters: {'encode_dim': 64.0, 'nn_width': 96.0, 'dropout_ae': 0.1, 'dropout_input': 0.15000000000000002, 'dropout_hidden': 0.25, 'dropout_output': 0.1, 'weight_decay': -5, 'pct_start': 0.30000000000000004, 'max_lr': 0.4}. Best is trial 13 with value: 0.6851402521133423.



best_valid_loss: 0.685600221157074 - best_valid_metric: -2797.328006461778


***